<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/amazon_product_selection_toys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [20]:
#1 Selected toys data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_toys = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print((df_toys.count(), len(df_toys.columns)))

(4864249, 15)


In [21]:
# Create a table to count data rows per year
toy_product_count = df_toys.groupby("product_id").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "reviews_count")
toy_product_count.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B00VTE0VJQ|            1|
|B00B28IQ50|          150|
|B002IMMDN0|          411|
|B00VJKTLMS|           13|
|B00YH60L7U|           19|
|B00WJ1O76G|           21|
|B00TJ2RBGQ|            2|
|B008MTYTI0|           37|
|B013TODK3W|            1|
|B000FNTGZC|          140|
|B01197XC02|            1|
|B00MWJIP7M|            3|
|B00WHXSDVQ|            2|
|B008GPMF9U|          321|
|B00146LV7A|           20|
|B000UUA988|           25|
|B000PRRRR2|            1|
|B00PMGLB7I|           51|
|B003AKOYPE|            1|
|B0066LCY0G|            5|
+----------+-------------+
only showing top 20 rows



In [28]:
toys_filtered = toy_product_count.filter(toy_product_count.reviews_count > 5000)
toys_filtered.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B004S8F7QM|        24281|
|B005JFNE8G|         6060|
+----------+-------------+

